In [1]:
import sys
sys.path.insert(1, '/Users/mac/Desktop/PycharmProjects/TAADL/src')
sys.path.insert(2, '/Users/mac/Desktop/PycharmProjects/TAADL/models')

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import json

from torch import optim
from network import GP_RNN, GPCopulaRNN
from config import DATA_PATH, E_DICT
from utils import transform, inv_transform, train_test_split, load_data

In [16]:
x_tr, cdf  = transform(z_tr)

In [3]:
#TODO add mean and covariance to existing RP strategy and do the plotting
params_GP    = torch.load('/Users/mac/Desktop/PycharmProjects/TAADL/models/GP_RNN.pt')
params_GPCop = torch.load('/Users/mac/Desktop/PycharmProjects/TAADL/models/GaussCopula.pt')

In [5]:
# load parameters for model and optimizer 
model     = GP_RNN(input_size=1, hidden_size=4, num_layers=2, rank_size=3, 
                        batch_size=3, num_asset=7, dropout=0.1)
model_cop = GPCopulaRNN(input_size=1, hidden_size=4, num_layers=2, rank_size=3, 
                        batch_size=3, num_asset=7, dropout=0.1, cdfs=cdf)

model.load_state_dict(params_GP['net_params'])
model_cop.load_state_dict(params_GPCop['net_params'])

<All keys matched successfully>

In [49]:
context_len = 18
z_preds  = []
gr_truth = []
z_stds    = [] 
for idx in range(context_len, z_tr.size(0)):
    feature, target = z_tr[idx-context_len:idx,:], z_tr[idx,:] 
    z_pred, z_std = model_cop.predict(feature, 10)
    
    # store values
    z_preds.append(z_pred.view(-1))
    z_stds.append(z_std.view(-1))
    gr_truth.append(target.view(-1))

z_preds   = torch.stack(z_preds)
z_stds    = torch.stack(z_stds)
gr_truth = torch.stack(gr_truth)

In [50]:
df_z        = pd.DataFrame(z_preds.numpy(), columns=l_ret.columns, index=l_ret.iloc[18:144].index)
df_z_std    = pd.DataFrame(z_stds.numpy(), columns=l_ret.columns, index=l_ret.iloc[18:144].index)
df_gr_truth = pd.DataFrame(gr_truth.numpy(), columns=l_ret.columns, index=l_ret.iloc[18:144].index)

In [51]:
df_z

,IEV,EEM,AGG,IEF,IYR,IAU,ITOT
Date,,,,,,,
2006-07-31,-0.005519,-0.009565,-0.000074,-0.001553,0.001102,-0.009316,0.002178
2006-08-31,-0.005523,-0.009568,-0.000074,-0.001553,0.001101,-0.009316,0.002178
2006-09-30,-0.005517,-0.009561,-0.000074,-0.001553,0.001096,-0.009312,0.002178
2006-10-31,-0.005517,-0.009571,-0.000074,-0.001554,0.001093,-0.009309,0.002178
2006-11-30,-0.005517,-0.009577,-0.000074,-0.001553,0.001102,-0.009317,0.002178
...,...,...,...,...,...,...,...
2016-08-31,-0.005524,-0.009565,-0.000074,-0.001554,0.001102,-0.009311,0.002178
2016-09-30,-0.005520,-0.009571,-0.000074,-0.001553,0.001096,-0.009310,0.002178
2016-10-31,-0.005516,-0.009573,-0.000074,-0.001553,0.001101,-0.009309,0.002178


In [52]:
df_gr_truth

,IEV,EEM,AGG,IEF,IYR,IAU,ITOT
Date,,,,,,,
2006-07-31,0.019513,0.023159,0.013794,0.016634,0.035841,0.032616,0.000880
2006-08-31,0.029616,0.015693,0.015785,0.017482,0.029355,-0.014971,0.021231
2006-09-30,0.006648,-0.008745,0.010064,0.012003,0.023532,-0.045970,0.024112
2006-10-31,0.038189,0.068200,0.006833,0.005370,0.062796,0.010196,0.032218
2006-11-30,0.035246,0.058123,0.010564,0.013029,0.045219,0.068915,0.021103
...,...,...,...,...,...,...,...
2016-08-31,0.004866,0.008798,-0.002157,-0.010166,-0.034063,-0.032790,0.003025
2016-09-30,0.006113,0.024873,0.000516,0.002371,-0.014837,0.007117,0.001879
2016-10-31,-0.032257,-0.008312,-0.008205,-0.015059,-0.050876,-0.029590,-0.023079


In [ ]:
if len(z_t.size()) == 2:
            z_t = z_t.unsqueeze(2)

        zs = []
        with torch.no_grad():
            for _ in range(num_samples):
                mu_t, _, _ = self.forward(z_t, pred=True)
                # use sampled z hat for next step prediction
                z_pred, _, _ = self.forward(mu_t[-1].unsqueeze(0), pred=True)
                zs.append(z_pred)
        zs = torch.stack(zs)
        z_mean, z_std = zs.mean(axis=0), zs.std(axis=0)
        return z_mean, z_std